In [1]:
import pandas as pd 
df_shopee = pd.read_csv('clothes_ruten/露天外套女.csv',index_col=0)  

In [2]:
#將csv data存到txt
f = open('shopee.txt','w')
for item in df_shopee['item']:
    print(item, file = f)
f.close()

In [3]:
def get_TF(words, topK=10): 
    tf_dic = {} 
    for w in words: 
        tf_dic[w] = tf_dic.get(w, 0) + 1 
    return sorted(tf_dic.items(), key = lambda x: x[1], reverse=True)[:topK]

In [4]:
def get_content(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as f: 
        content = '' 
        for l in f:
            l = l.strip() 
            content += l 
    return content

In [5]:
stop_words = []
stop_words.append(' ')
stop_words.append('【')
stop_words.append('】')
stop_words.append('-')
stop_words.append('╯')
stop_words.append('❤')
stop_words.append('♡')
stop_words.append('(')
stop_words.append('！')
stop_words.append(')')
stop_words.append('~')
stop_words.append('📣')
stop_words.append('👗')
stop_words.append('/')
stop_words.append('*')
stop_words.append('☆')
stop_words.append('.')
stop_words.append('♥')
stop_words.append('★')
stop_words.append('+')
stop_words.append('》')
stop_words.append('[')
stop_words.append(']')
stop_words.append('Ω')
stop_words.append('《')
stop_words.append('、')
stop_words.append('§')
stop_words.append('🔥')
stop_words.append('💥')
stop_words.append('🎀')
stop_words.append('!')
stop_words.append('^')
stop_words.append('💖')
stop_words.append('≡')
stop_words.append(',')
stop_words.append('_')
stop_words.append('＊')
stop_words.append('&')
stop_words.append('~')
stop_words.append('#')
stop_words.append('～')
stop_words.append('，')
stop_words.append(':')
stop_words.append('◎')
stop_words.append('』')
stop_words.append('『')
stop_words.append('|')
stop_words.append('@')
stop_words.append('├')
stop_words.append('．')
stop_words.append('.....')
stop_words.append('=')

In [6]:
import glob 
import jieba 
files = 'shopee.txt'
corpus = [get_content(files)]
split_words = [x for x in jieba.cut(corpus[0]) if x not in stop_words]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tz/dqn0619n7v925j6hb3s0whvm0000gn/T/jieba.cache
Loading model cost 0.836 seconds.
Prefix dict has been built successfully.


In [7]:
print('樣本的topK(10)詞：'+str(get_TF(split_words,100)))

樣本的topK(10)詞：[('外套', 5924), ('男女', 1353), ('連帽', 1042), ('女', 657), ('風衣', 614), ('款', 604), ('新款', 598), ('長', 541), ('夾克', 541), ('韓版', 515), ('運動', 496), ('袖', 451), ('防風', 451), ('情侶', 436), ('保暖', 419), ('休閒', 396), ('秋冬', 374), ('衛衣', 362), ('T', 355), ('加厚', 354), ('寬', 349), ('女裝', 342), ('鬆', 341), ('修身', 326), ('薄', 319), ('衣服', 309), ('棒球', 307), ('上衣', 299), ('愛迪達', 296), ('防曬', 294), ('開衫', 288), ('小', 281), ('女款', 269), ('男', 264), ('中長', 244), ('大衣', 234), ('女生', 232), ('現貨', 226), ('羽絨', 226), ('女童', 216), ('牛仔', 209), ('色', 205), ('防水', 204), ('短款', 204), ('免運', 201), ('加絨', 200), ('透氣', 192), ('衣', 192), ('裝', 191), ('碼', 190), ('Adidas', 187), ('背心', 184), ('帽', 182), ('春秋', 178), ('可', 175), ('套裝', 174), ('薄款', 170), ('衫', 161), ('搭', 160), ('刷毛', 159), ('風', 153), ('學生', 151), ('大', 148), ('穿', 148), ('毛衣', 140), ('披肩', 137), ('版', 136), ('二次元', 136), ('周邊', 136), ('2', 135), ('純棉', 134), ('服', 133), ('襯衫', 133), ('夏季', 131), ('衝鋒衣', 129), ('冬季', 129), ('防曬衣', 128)

In [8]:
frequent_array = get_TF(split_words,100)
score = [0]*len(df_shopee['item'])
index = 0
for item in df_shopee['item']:
    for i in range(len(frequent_array)):
        if (frequent_array[i][0]) in str(item):
            score[index] += frequent_array[i][1]
    index += 1
df_shopee['item'] = score

In [9]:
price_array = df_shopee['price']
price_array = list(map(str, price_array))
price_array_2 = ['0'] * len(price_array)
index_i = 0
for item in price_array:
    if('-' in item):
        index_j = str.index(item,'-')
        price_array_2[index_i] = item[-index_j:]
    else:
        price_array_2[index_i] = item
    index_i += 1
index_i = 0
for item in price_array_2:
    price_array_2[index_i] = str.replace(item,"$","")
    price_array_2[index_i] = str.replace(price_array_2[index_i],",","")
    index_i += 1
df_shopee['price'] = price_array_2

In [10]:
a = df_shopee['price']
a = list(map(float,a))
a = list(map(int,a))
df_shopee['price'] = a

In [11]:
df_shopee = df_shopee.drop(df_shopee[df_shopee.price > 2000].index)
df_shopee = df_shopee.drop(df_shopee[df_shopee.num_sold > 5000].index)

In [12]:
df_shopee.describe()

,item,rate,num_product,num_sold,price
count,3423.000000,3423.000000,3.423000e+03,3423.000000,3423.000000
mean,10190.809232,3863.153082,4.187469e+04,23.995326,713.886941
std,2595.353368,9120.421765,1.167987e+05,162.475135,419.858877
min,0.000000,0.000000,0.000000e+00,1.000000,0.000000
25%,9212.500000,242.000000,5.465000e+02,1.000000,398.000000
50%,10509.000000,1110.000000,1.797000e+03,2.000000,629.000000
75%,11733.500000,3983.000000,1.967500e+04,5.000000,900.000000
max,16630.000000,229877.000000,1.434136e+06,4575.000000,2000.000000


In [13]:
result_baseline = 20/df_shopee['num_sold'].max()
num_sold_max = df_shopee['num_sold'].max()
item_max = df_shopee['item'].max()
print(result_baseline)
print(num_sold_max)
print(item_max)

0.004371584699453552
4575
16630


In [14]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sc.fit(df_shopee)
df_shopee = pd.DataFrame(sc.transform(df_shopee), index=df_shopee.index, columns=df_shopee.columns)

In [15]:
score = 0
title_a = '外套男女連帽女'
for i in range(len(frequent_array)):
        if (frequent_array[i][0]) in title_a:
            score+= frequent_array[i][1]
score/=item_max
print(score)
import numpy as np
x_final = np.array([[score,df_shopee['rate'].mean(),
df_shopee['num_product'].mean(),
df_shopee['price'].mean()]])

0.572699939867709


In [16]:
df_shopee.describe()

,item,rate,num_product,num_sold,price
count,3423.000000,3423.000000,3423.000000,3423.000000,3423.000000
mean,0.612797,0.016805,0.029199,0.005027,0.356943
std,0.156065,0.039675,0.081442,0.035521,0.209929
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.553969,0.001053,0.000381,0.000000,0.199000
50%,0.631930,0.004829,0.001253,0.000219,0.314500
75%,0.705562,0.017327,0.013719,0.000875,0.450000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
len_data = len(df_shopee)-200
len_data

3223

In [46]:
df = df_shopee
from sklearn.utils import shuffle
df = shuffle(df)

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns = ['num_sold']).values[0:len_data],
    df['num_sold'].values[0:len_data], test_size=0.5)
X_ans = df.drop(columns = ['num_sold']).values[len_data:]

In [48]:
import xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.04
params['max_depth'] = 8
params['learning_rate'] = 0.01

d_train = xgboost.DMatrix(X_train, label=y_train)
d_valid = xgboost.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgboost.train(params, d_train, 1000, watchlist, early_stopping_rounds=100, verbose_eval=10)
y_pred = bst.predict(xgboost.DMatrix(X_test))

[0]	train-logloss:0.68344	valid-logloss:0.68342
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.59610	valid-logloss:0.59585
[20]	train-logloss:0.52336	valid-logloss:0.52291
[30]	train-logloss:0.46201	valid-logloss:0.46136
[40]	train-logloss:0.40971	valid-logloss:0.40887
[50]	train-logloss:0.36477	valid-logloss:0.36376
[60]	train-logloss:0.32589	valid-logloss:0.32471
[70]	train-logloss:0.29204	valid-logloss:0.29072
[80]	train-logloss:0.26244	valid-logloss:0.26099
[90]	train-logloss:0.23647	valid-logloss:0.23489
[100]	train-logloss:0.21360	valid-logloss:0.21190
[110]	train-logloss:0.19341	valid-logloss:0.19159
[120]	train-logloss:0.17555	valid-logloss:0.17360
[130]	train-logloss:0.15970	valid-logloss:0.15765
[140]	train-logloss:0.14556	valid-logloss:0.14356
[150]	train-logloss:0.13294	valid-logloss:0.13101
[160]	train-logloss:0.12168	valid-logloss:0.11981
[170]	tra

In [49]:
ans_pred1 = bst.predict(xgboost.DMatrix(X_ans))

In [50]:
answer = df['num_sold'].values[len_data:]
count = 0 
for i in range(len(answer)):
        if(abs(answer[i]-ans_pred1[i])<=result_baseline):
            count += 1
a = len(df['num_sold'][len_data:])
print(str(len_data+200)+'/'+str(count/a))

3423/0.815


In [51]:
ans_pred1.mean() * num_sold_max

22.473529796116054

In [52]:
ans_pred2 = bst.predict(xgboost.DMatrix(x_final))
ans_pred2[0] * num_sold_max

5.156280496157706